In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request
import os
import urllib.parse
import shutil
from io import BytesIO

#import cv2
from PIL import Image
import time
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


In [2]:
fakeBlogKeywordList = ["스토리앤","seoulouba","mateb.kr","revu","weble","ohmyblog","mrblog","tble",
    "dinnerqueen","%EA%B3%B5%EC%A0%95%EA%B1%B0%EB%9E%98%EC%9C%84%EC%9B%90%ED%9A%8C-%EB%AC%B8%EA%B5%AC",
    "%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9B%90%EC%A0%95%EB%8C%80",
    "banner_","%EC%9D%B4%EC%8A%88%EB%B8%94%EB%A1%9C%EA%B7%B8",
    "%EB%B0%B0%EB%84%88","http://echoblog.net/images/sponsor-banner.png",
                      "sponsor","banner","echoblog"]

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

#define const value
batch_size = 6
epochs = 10
IMG_HEIGHT = 120
IMG_WIDTH = 240


In [4]:
def create_model():
    model = Sequential([
        Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
        MaxPooling2D(),
        Conv2D(32, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Conv2D(64, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [5]:
#load model
checkpoint_path = "model_0716_v1(batch-4)/cp-0010.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# create model
model = create_model()
model.load_weights(latest)

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
np.set_printoptions(suppress=True)

model_0716_v1(batch-4)/cp-0010.ckpt


In [6]:
def convertToJpeg(im):
    with BytesIO() as f:
        new_img = im.convert("RGB")
        new_img.save(f, format='JPEG')
        return f.getvalue()


In [7]:
def PredictionWithoutDownload(uri):

    start_time = time.time();
    filename = uri.split('/')[-1].split('?')[0];
    img_url2 = urllib.parse.quote_plus(str(filename))
    img_url_final = uri.replace(uri.split('/')[-1].split('?')[0],img_url2)
    
#     response = requests.get(uri)
    response = urllib.request.urlopen(img_url_final).read()
#     print("---[1] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
#     img = Image.open(BytesIO(response.content))
    img = Image.open(BytesIO(response))
    
    if(img.format == "PNG") or (img.format == "GIF"):
        print("this image is",img.format,"type. could be change to jpg")
        png_to_jpg = convertToJpeg(img)
        img = Image.open(BytesIO(png_to_jpg))
#         img.show()
#     print("---[2] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #width,height
    resize_img = img.resize((IMG_WIDTH, IMG_HEIGHT))
#     print("---[3] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();

    input_arr = tf.keras.preprocessing.image.img_to_array(resize_img)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
#     print("---[4] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    
    predictions_class = model.predict_classes(input_arr)
#     print("---[5] {}s seconds---".format((time.time()-start_time),".2f"))

    print("========pridiction[Without Download]========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    if(predictions_class == [[0]]):
        predictions = model.predict(input_arr)
        print("predictions : ",predictions);

        print("this is fake blog")
        saveImageVol2(uri,predictions)
#     print("predictions : " , predictions)
    return predictions_class


In [8]:
# =====================================================================
#이미지 저장만을 위한 함수
def saveImageVol2(img_url,predict):
    list = os.listdir("ad_img")
    last_index = len(list)
#         print("last_index : ",last_index)
    dirPath = 'ad_img'
    imgPath = f'ad_img/{last_index}_{predict}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
    except Exception as ex:
        print("img save error : " , ex)
        return False

In [9]:
# =====================================================================
#이미지 분류 [my model]

def imageClassification(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
#    print("input_arr type : " ,type(input_arr))
    input_arr = np.array([input_arr])  # Convert single image to a batch.

    predictions_class = model.predict_classes(input_arr)
    predictions = model.predict(input_arr)

    print("========pridiction========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    print("predictions : " , predictions);
#    print("predictions argmax : ",np.argmax(predictions[0])); # same way
    return predictions_class,predictions;

# =====================================================================
#이미지 저장
def saveImage(div_option,index,img_url,nickName,postNumber):
    #블로그 유저 별로 폴더를 나눌건지에 대한 옵션
    #true -> path : image/{nickName}/{postNumber}
    #false -> path : image/myimg/
    if(div_option):
        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"        
        imgPath = f'image/{nickName}/{postNumber}/{index}.jpg'
        dirPath = f'image/{nickName}/{postNumber}'
    #폴더별로 분기하지 않을떄 path : image/myimg/
    else:
        list = os.listdir("image/temp")
        last_index = len(list)
#         print("last_index : ",last_index)
        dirPath = 'image/temp'
        imgPath = f'image/temp/{last_index}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
                
    #파일 저장
    #temp에 저장 -> classification -> 분류 저장
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
#         print("img_url_final : " ,img_url_final);
        for item in fakeBlogKeywordList:
            if(img_url_final.find(item) > 0):
                print('this is fake img')

                imgPath = f"image/temp/fake_{last_index}.jpg"
                break;
        
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        my_prediction, score  = imageClassification(imgPath);
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
        
        if (my_prediction[0] == 0):
            list = os.listdir("ad_img")
            last_index = len(list)
            #os.rename(imgPath, f'ad_img/{last_index}.jpg')
            #shutil.move(imgPath, f'ad_img/{last_index}.jpg')
            if(imgPath.find("fake_") > 0):  #모델, url 둘다 이미지로 판단
                os.replace(imgPath, f'ad_img/url_model_{last_index}_{score}.jpg')
            else: #모델만 이미지로 판단
                os.replace(imgPath, f'ad_img/model_{last_index}_{score}.jpg')
            return False
        else: # url만 이미지로 판단
            list = os.listdir("not_fake")
            last_index = len(list)
            if(imgPath.find("fake_") > 0):
                list = os.listdir("ad_img")
                last_index = len(list)
                os.replace(imgPath, f'not_fake/url_{last_index}_{score}.jpg')
            else: #모델, url 둘다 이미지로 판단
                if(score <= 1500):  #confidence가 1500이하일때만 저장 
                    os.replace(imgPath, f'not_fake/url_model_{last_index}_{score}.jpg')
        return True
    except Exception as ex:
        print("img save error : " , ex)
        return False
    
    
# =====================================================================
# image gethering
def findBlogImgGethering(searchQuery, searchOption, page):
    try:
        start_time = time.time();

        url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
        html = requests.get(url)
        # 1차, blog URL 찾기
        soup = BeautifulSoup(html.text, 'html.parser')
        index = 0;

        url_list = [];
        for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
            child_item = li_item.find('a',{'class' : 'sh_blog_title'})
            title = child_item.attrs['title']
            href = child_item.attrs['href']

            #url list 저장
            url_list.append(href)

            print(f'title : {title} , href : {href}')
            print('----------------------------------')

        # url 파싱 및 예외처리
        for url_item in url_list:
            nickName = "";
            postNumber = "";

            if(url_item.find("blog.me") > 0):
                parsing = url_item;
                parsing = parsing.replace("https://","")
                nickName = parsing.split('.')[0]
                postNumber = parsing.split('/')[1]
                blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber

            else:
                blogUrl = url_item.replace("https://", "https://m.");

            nickName = blogUrl.split('/')[-1].split('?')[0]
            postNumber = blogUrl.split('/')[-1].split('?')[1].split('=')[-1]

            if nickName is "":
                nickName = "unknown"
            if postNumber is "":
                postNumber = "1"

            print("------ digging more -------")
            print("nickName : " , blogUrl.split('/')[-1].split('?')[0])
            print("postNumber : " , blogUrl.split('/')[-1].split('?')[1].split('=')[-1])
            print("blogUrl : " , blogUrl)        
            start_time = time.time();
            #전체 목록 순회
            blog_html = requests.get(blogUrl)
            blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
            blog_image_class = blog_soup.find_all('div',{'class' : 'se-image'});

            for div_obj in blog_image_class:
                for idx, img_item in enumerate(div_obj.find_all('img')):
                    img_url = img_item.attrs['src'];

                    if(img_url.find("w80_blur") > 0):
                        img_url = img_url.replace("w80_blur" , "w800")
                        #np array after save image(for crawling data)
                    isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
                    #np arr from img_url for validation
#                     PredictionWithoutDownload(img_url)
                    if(isSaveSuccess):  
                        index = index+1
    #         print("---{}s seconds---".format(time.time()-start_time),".2f")
    except Exception as ex:
        print("error catch : " , ex)
        print("url : " ,img_url)


In [10]:
searchQuery = "서울대입구역 피자집"
searchOption = 'sim'  #sim or date  
page = 1
end_page = 10
for i in list(range(page,end_page)):
    if i>1:
        page = page + 10;
    findBlogImgGethering(searchQuery,searchOption, page)
# findBlogImgGethering(searchQuery,searchOption,page)

title : 서울대입구역 피자 샤로수길 갓성비 피자집 피자보이시나 , href : https://blog.naver.com/hs486mh?Redirect=Log&logNo=221683003524
----------------------------------
title : 서울대입구역 피자집 : 피자네버슬립스 ☆ , href : https://blog.naver.com/1john324?Redirect=Log&logNo=221707130754
----------------------------------
title : 서울대입구역 피자, 피자보이시나(샤로수길 피자집) 맛보시게나 (메뉴&가격) , href : https://blog.naver.com/ji_ram_jiral?Redirect=Log&logNo=222018690249
----------------------------------
title : 서울대입구역, 샤로수길 피자집을 찾았다! (tailor slice pizza) , href : https://blog.naver.com/7474562?Redirect=Log&logNo=221385130551
----------------------------------
title : 서울대입구역, 샤로수길 피자집 추천!!! (ft.피자네버슬립스) , href : https://blog.naver.com/yedeng_2?Redirect=Log&logNo=221572054195
----------------------------------
title : 서울대입구역 샤로수길 데이트 : 독립서점 살롱드북, 피자집 파고, 카페 미드레벨 , href : https://blog.naver.com/areum_land?Redirect=Log&logNo=221880263306
----------------------------------
title : [ 서울대입구역 맛집 ] 이태리파파 , href : https://blog.naver.com/sss_sr?Redirect=Log&

========pridiction========
predictions class :  [[1]]
predictions :  [[4913.9937]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3280.8599]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4962.226]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5256.677]]
------ digging more -------
nickName :  areum_land
postNumber :  221880263306
blogUrl :  https://m.blog.naver.com/areum_land?Redirect=Log&logNo=221880263306
========pridiction========
predictions class :  [[1]]
predictions :  [[3137.4158]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4864.6147]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4881.8047]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4064.1807]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1254.6649]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3868.157]]
========pridicti

========pridiction========
predictions class :  [[1]]
predictions :  [[5830.6294]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2775.6062]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4355.8022]]
title : 서울대입구역 샤로수길 [피자보이시나 서울대점] , href : https://soomin4745.blog.me/221771875156
----------------------------------
title : 서울대입구역 맛집 피자네버슬립스에서 데이트 , href : https://blog.naver.com/pretty_mr24?Redirect=Log&logNo=221637458257
----------------------------------
title : 샤로수길 피자집, 피자보이시나 서울대점 다녀왔어요~ , href : https://blog.naver.com/starry_life?Redirect=Log&logNo=221656009732
----------------------------------
title : 서울대입구역 피자+파스타 맛집 샤로수길"피자보이시나" , href : https://blog.naver.com/skwjddls2?Redirect=Log&logNo=221709689705
----------------------------------
title : 서울대입구역 피자 샤로수길 피자 먹자 , href : https://blog.naver.com/lklk332?Redirect=Log&logNo=221994500340
----------------------------------
title : [샤로수길 맛집] 피자와 파스타가 맛있는 '피자보이시나' 서울대입구역 맛집 , href : 

========pridiction========
predictions class :  [[1]]
predictions :  [[1731.2633]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2361.4485]]
========pridiction========
predictions class :  [[1]]
predictions :  [[111.117805]]
========pridiction========
predictions class :  [[1]]
predictions :  [[366.1507]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-8.452893]]
this is fake img
========pridiction========
predictions class :  [[0]]
predictions :  [[-3189.024]]
------ digging more -------
nickName :  skwjddls2
postNumber :  221709689705
blogUrl :  https://m.blog.naver.com/skwjddls2?Redirect=Log&logNo=221709689705
========pridiction========
predictions class :  [[1]]
predictions :  [[1353.47]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5811.5425]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5999.314]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1709.522]]
==

========pridiction========
predictions class :  [[1]]
predictions :  [[2863.8252]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2009.195]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3774.6604]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3750.0427]]
------ digging more -------
nickName :  dnjsxo09
postNumber :  221952730769
blogUrl :  https://m.blog.naver.com/dnjsxo09?Redirect=Log&logNo=221952730769
========pridiction========
predictions class :  [[1]]
predictions :  [[3027.8962]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2922.9758]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1124.4816]]
========pridiction========
predictions class :  [[1]]
predictions :  [[960.30396]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2442.2898]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1005.92883]]
========pridictio

...

========pridiction========
predictions class :  [[0]]
predictions :  [[-52.943142]]
========pridiction========
predictions class :  [[1]]
predictions :  [[888.53284]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1165.3258]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2649.5168]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3304.762]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2948.7307]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5689.6206]]
========pridiction========
predictions class :  [[1]]
predictions :  [[6256.495]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2673.6782]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4019.956]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4667.1597]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3016.4705]]
======

========pridiction========
predictions class :  [[1]]
predictions :  [[6239.1074]]
this is fake img
========pridiction========
predictions class :  [[0]]
predictions :  [[-2045.601]]
------ digging more -------
nickName :  o_guerim
postNumber :  221489782213
blogUrl :  https://m.blog.naver.com/o_guerim?Redirect=Log&logNo=221489782213
========pridiction========
predictions class :  [[1]]
predictions :  [[2811.9219]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3857.8794]]
------ digging more -------
nickName :  rhkrghksgml
postNumber :  221333611988
blogUrl :  https://m.blog.naver.com/rhkrghksgml?Redirect=Log&logNo=221333611988
------ digging more -------
nickName :  blueyuns
postNumber :  220242641221
blogUrl :  https://m.blog.naver.com/blueyuns?Redirect=Log&logNo=220242641221
------ digging more -------
nickName :  pie907
postNumber :  221563630074
blogUrl :  https://m.blog.naver.com/pie907?Redirect=Log&logNo=221563630074
========pridiction========
prediction

========pridiction========
predictions class :  [[1]]
predictions :  [[2565.1084]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2731.4514]]
------ digging more -------
nickName :  threeseul28
postNumber :  221359550987
blogUrl :  https://m.blog.naver.com/threeseul28?Redirect=Log&logNo=221359550987
------ digging more -------
nickName :  jennyleekr
postNumber :  221113264793
blogUrl :  https://m.blog.naver.com/jennyleekr?Redirect=Log&logNo=221113264793
------ digging more -------
nickName :  flyyunmin
postNumber :  221412454914
blogUrl :  https://m.blog.naver.com/flyyunmin?Redirect=Log&logNo=221412454914
------ digging more -------
nickName :  mrkim1988
postNumber :  221412407023
blogUrl :  https://m.blog.naver.com/mrkim1988?Redirect=Log&logNo=221412407023
------ digging more -------
nickName :  0415carrot
postNumber :  221436201854
blogUrl :  https://m.blog.naver.com/0415carrot?Redirect=Log&logNo=221436201854
========pridiction========
predictions class :  [[1

========pridiction========
predictions class :  [[1]]
predictions :  [[2234.9097]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2345.453]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2490.0403]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3392.75]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1350.0394]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1657.721]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5287.591]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5055.4873]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3403.3452]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2248.8848]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3024.5718]]
========pridiction========
predictions class :  [[1]]
predictions :  [[7444.963]]
========pr

========pridiction========
predictions class :  [[1]]
predictions :  [[5965.361]]
========pridiction========
predictions class :  [[1]]
predictions :  [[5119.3613]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3317.9]]
========pridiction========
predictions class :  [[1]]
predictions :  [[4051.5403]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2678.5747]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3902.6235]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3155.1458]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3304.1904]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2860.0596]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3856.807]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3479.2554]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2708.8052]]
========p

In [11]:
# # blog gethering test
# def findBlogListTest(searchQuery, searchOption, page):
#     try:
#         print(f'\n###############{page}###############\n');
#         url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
#         html = requests.get(url)
#         # 1차, blog URL 찾기
#         soup = BeautifulSoup(html.text, 'html.parser')
#         index = 0;

#         url_list = [];
#         for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
#             child_item = li_item.find('a',{'class' : 'sh_blog_title'})
#             title = child_item.attrs['title']
#             href = child_item.attrs['href']

#             #url list 저장
#             url_list.append(href)

#             print(f'title : {title} , href : {href}')
#             print('----------------------------------')

#     except Exception as ex:
#         print("error catch : " , ex)
#         print("url : " ,img_url)

In [12]:
# searchQuery = "서울대입구역 하남돼지집"
# searchOption = 'sim'  #sim or date  
# page = 1
# end_page = 11
# # 1->11->21->순으로
# for i in list(range(page,end_page)):
#     if i>1:
#         page = page + 10;
#     findBlogListTest(searchQuery,searchOption,page)